# **PREPROCESSING**
## *Marc Fuentes i Víctor Novelle*

*December 2020*

The first step in the data preprocessing consisted of preparing the environment to work comfortably.

In [95]:
# Necessary libraries for code execution.
import pandas as pd
import altair as alt

In [96]:
# Google drive loading as work station for local-usage of the files.
from google.colab import drive
drive.mount('/content/gdrive',force_remount= True)

Mounted at /content/gdrive


Once we have prepared the environment, we proceeded to load our raw data and start cleaning it. 

In order to do this project, we have decided to use the previously selected dataframe, which consisted of the *Bicing®* stations data for October,2019.

In [97]:
# Loading of the selected data set.
df = pd.read_csv('/content/gdrive/My Drive/GCED/Q5/VI/Projecte 2/octubre19.csv')
# Observation of the first values of our data set.
df.head()

,station_id,num_bikes_available,num_bikes_available_types.mechanical,num_bikes_available_types.ebike,num_docks_available,is_installed,is_renting,is_returning,last_reported,is_charging_station,status,last_updated,ttl
0,1,25,24,1,4,1,1,1,1.569881e+09,True,IN_SERVICE,1569880820,30
1,2,12,11,1,15,1,1,1,1.569881e+09,True,IN_SERVICE,1569880820,30
2,3,19,19,0,8,1,1,1,1.569881e+09,True,IN_SERVICE,1569880820,30
3,4,15,15,0,3,1,1,1,1.569881e+09,True,IN_SERVICE,1569880820,30
4,5,35,35,0,3,1,1,1,1.569881e+09,True,IN_SERVICE,1569880820,30


The next step after loading the raw data was to convert the different temporal variables to a more generic format (YYYY-MM-dd hh:mm:ss).

In [98]:
# Changing the temporal format.
df['last_reported'] = pd.to_datetime(df['last_reported'],unit = 's')
df['last_updated'] = pd.to_datetime(df['last_updated'],unit = 's')

In [99]:
# Checking that the transformation was correctly applied.
df.head()

,station_id,num_bikes_available,num_bikes_available_types.mechanical,num_bikes_available_types.ebike,num_docks_available,is_installed,is_renting,is_returning,last_reported,is_charging_station,status,last_updated,ttl
0,1,25,24,1,4,1,1,1,2019-09-30 22:00:01,True,IN_SERVICE,2019-09-30 22:00:20,30
1,2,12,11,1,15,1,1,1,2019-09-30 21:59:14,True,IN_SERVICE,2019-09-30 22:00:20,30
2,3,19,19,0,8,1,1,1,2019-09-30 21:59:47,True,IN_SERVICE,2019-09-30 22:00:20,30
3,4,15,15,0,3,1,1,1,2019-09-30 21:58:23,True,IN_SERVICE,2019-09-30 22:00:20,30
4,5,35,35,0,3,1,1,1,2019-09-30 21:56:25,True,IN_SERVICE,2019-09-30 22:00:20,30


The next step consisted in performing an integrity analysis. In this, we studied if for some stations/time was any problem for the data collected during the month. This process was divided into several steps:

### 1. Installation
Firstly, we checked if some stations were not correctly installed during some period throughout the month.


In [100]:
#Checking if the stations are correctly installed.
print(sum(df['is_installed'] != True))

df_clean = df[df['is_installed'] == True]

32872


As it can be seen, $32872$ rows of our raw dataframe indicate that the stations were not properly installed at that moment. As the original number of rows is humongous ($3655823$), we have decided to eliminate those.


---
### 2. Returning

Next, we proceeded to analyze if all the *Bicing®* stations were able to return the bikes as they should.

In [101]:
#Checking if the stations allow to return the bikes.
print(sum(df_clean['is_returning'] != True))

df_clean = df_clean[df_clean['is_returning'] == True]

4221


As the number of rows that do not properly return the bikes is small in comparison to the dimensions of the dataset, we also decided to delete them.

---
### 3. Renting

Then, besides just checking if the returnings were working fine, we also scanned the data frame to find moments where the stations did not allow the users to rent.

In [102]:
#Checking if the stations were renting properly.
print(sum(df_clean['is_renting'] != True))

df_clean = df_clean[df_clean['is_renting'] == True]

2


As only two rows were affected by this problem they were removed.

---
### 4. Is charging station? 

In this section, we checked if some of the stations did not allow electrical bikes. As can be seen, that was not the case.

In [103]:
# Checking is some station was not a charging station.
print(sum(df_clean["is_charging_station"] != True))

0


---
### 5. Status

Analyzing if any row has a status different of "IN SERVICE" (indicating that there is some problem at that time for that station). This was also not the case.

In [104]:
# Checking the status of all the stations.
sum(df_clean['status'] != "IN_SERVICE")

0

As we have cleaned all the columns, now they only contain values that show that all the stations are behaving properly. Therefore, these variables can be  deleted since they do not provide information of interest.



In [105]:
df_clean = df_clean.drop(columns = ['is_installed','is_renting','is_returning','is_charging_station'])

---
Once the integrity check was finished, we proceeded to create new columns that would ease applying selections/aggregations further on this cleaning process and on the future visualizations.

Firstly, we created a column that stored the day when the update of the station info was performed, to later obtain the daily average of the desired metrics.

In [106]:
# Extracting the day of the year as a timestamp.
df_clean["day"] = df_clean["last_reported"].transform(lambda x: x.date())

Then we checked if all of the rows corresponded to the status of the stations in October or other months were wrongly included on this data frame.

In [107]:
#Checking that all the info belong to October.

import datetime 

lw = datetime.date(2019, 10, 1) 
up = datetime.date(2019, 10, 31) 

sum((df_clean["day"] < lw) | (df_clean["day"] > up))

10303

In [108]:
df_clean[(df_clean["day"] < lw) | (df_clean["day"] > up)].head()

,station_id,num_bikes_available,num_bikes_available_types.mechanical,num_bikes_available_types.ebike,num_docks_available,last_reported,status,last_updated,ttl,day
0,1,25,24,1,4,2019-09-30 22:00:01,IN_SERVICE,2019-09-30 22:00:20,30,2019-09-30
1,2,12,11,1,15,2019-09-30 21:59:14,IN_SERVICE,2019-09-30 22:00:20,30,2019-09-30
2,3,19,19,0,8,2019-09-30 21:59:47,IN_SERVICE,2019-09-30 22:00:20,30,2019-09-30
3,4,15,15,0,3,2019-09-30 21:58:23,IN_SERVICE,2019-09-30 22:00:20,30,2019-09-30
4,5,35,35,0,3,2019-09-30 21:56:25,IN_SERVICE,2019-09-30 22:00:20,30,2019-09-30


As can be seen, we had station status dating from September. As we are only focused on analyzing October values, we deleted those.

In [109]:
df_clean = df_clean[(df_clean["day"] >= lw) & (df_clean["day"] <= up)]

Next, we created two new columns, one that stores the day of the week for each register and the other one  stores the day of the month for each row. Even though this information could be extracted using *Altair* transformations functions we decided to store them as a separate variable, mainly to avoid code repetition. As this values will be used frequently, either as "visual encoded attribute" or "filtering attribute", reading them directly from the dataframe instead of applying an *Altair* transformation every time we need them will make it easier to read and understand the code.

In [110]:
# Storing the weekday and monthday of last reported
df_clean["weekday"] =  pd.to_datetime(df_clean.day).dt.dayofweek+1
df_clean["day_month"] = pd.to_datetime(df_clean.day).dt.day
df_clean.head()

,station_id,num_bikes_available,num_bikes_available_types.mechanical,num_bikes_available_types.ebike,num_docks_available,last_reported,status,last_updated,ttl,day,weekday,day_month
10250,1,27,26,1,2,2019-10-01 00:02:03,IN_SERVICE,2019-10-01 00:05:10,20,2019-10-01,2,1
10251,2,12,12,0,15,2019-10-01 00:01:01,IN_SERVICE,2019-10-01 00:05:10,20,2019-10-01,2,1
10252,3,18,18,0,9,2019-10-01 00:01:22,IN_SERVICE,2019-10-01 00:05:10,20,2019-10-01,2,1
10253,4,14,14,0,4,2019-10-01 00:04:38,IN_SERVICE,2019-10-01 00:05:10,20,2019-10-01,2,1
10254,5,36,35,1,2,2019-10-01 00:03:35,IN_SERVICE,2019-10-01 00:05:10,20,2019-10-01,2,1


In this step, we computed the daily average per station of the metrics we are interested in.

In [94]:
df_clean= df_clean.groupby(["station_id", "day","weekday","day_month"]).agg('mean')
df_clean.head()

num_bikes_available  ...  num_docks_available
station_id day        weekday day_month                       ...                     
1          2019-10-01 2       1                    12.533101  ...            14.937282
           2019-10-02 3       2                    13.615917  ...            13.079585
           2019-10-03 4       3                    14.159722  ...            13.159722
           2019-10-04 5       4                    12.836806  ...            16.677083
           2019-10-05 6       5                    18.298611  ...             9.916667

[5 rows x 4 columns]

After cleaning the data frame that provides information about the status of the *Bicing®* stations, we proceeded to load the data frame that contains the geospatial information and clean it too.

In [111]:
# Loading of the Bicing® geospatial info dataframe
st_info =  pd.read_csv('/content/gdrive/My Drive/GCED/Q5/VI/Projecte 2/bicing_station_districts.csv')
st_info.head()

,station_id,name,physical_configuration,lat,lon,altitude,address,post_code,capacity,Latlon,Barri,Districte,Municipi
0,36,"AV. DE LA CATEDRAL, 6",ELECTRICBIKESTATION,41.385062,2.176683,8,"AV. DE LA CATEDRAL, 6",8002,21,"41.38506160000001,2.1766834",el Gòtic,Ciutat Vella,Barcelona
1,53,PL. CARLES PI I SUNYER,ELECTRICBIKESTATION,41.385086,2.174016,10,PL. CARLES PI I SUNYER,8002,21,"41.385086,2.174016",el Gòtic,Ciutat Vella,Barcelona
2,55,"LA RAMBLA, 80",ELECTRICBIKESTATION,41.381428,2.173286,8,"LA RAMBLA, 80",8002,19,"41.3814279,2.1732861",el Gòtic,Ciutat Vella,Barcelona
3,57,"RAMBLA, 2",ELECTRICBIKESTATION,41.376876,2.177225,4,"RAMBLA, 2",8002,25,"41.3768761,2.1772251",el Gòtic,Ciutat Vella,Barcelona
4,126,PG. DE COLOM /VIA LAIETANA,ELECTRICBIKESTATION,41.380628,2.182192,4,PG. DE COLOM /VIA LAIETANA,8002,23,"41.380628,2.1821916",el Gòtic,Ciutat Vella,Barcelona


As we have done previously, we checked if some columns did not provide any information. In this case, as all stations are electrical, the "physical configuration" variable can be removed.

In [112]:
# Checking if some station is not electrical
sum(st_info['physical_configuration']!= "ELECTRICBIKESTATION")

st_info_clean = st_info.drop(columns = 'physical_configuration')

Then, the join of the two data frames is performed.

In [115]:
# Join of the two dataframes
df_clean = df_clean.join(st_info_clean.set_index('station_id'), on='station_id')
df_clean.head()

,station_id,num_bikes_available,num_bikes_available_types.mechanical,num_bikes_available_types.ebike,num_docks_available,last_reported,status,last_updated,ttl,day,weekday,day_month,name,lat,lon,altitude,address,post_code,capacity,Latlon,Barri,Districte,Municipi
10250,1,27,26,1,2,2019-10-01 00:02:03,IN_SERVICE,2019-10-01 00:05:10,20,2019-10-01,2,1,"GRAN VIA CORTS CATALANES, 760",41.397978,2.180107,16.0,"GRAN VIA CORTS CATALANES, 760",8013.0,46.0,"41.3979779,2.1801068999999997",el Fort Pienc,Eixample,Barcelona
10251,2,12,12,0,15,2019-10-01 00:01:01,IN_SERVICE,2019-10-01 00:05:10,20,2019-10-01,2,1,"C/ ROGER DE FLOR, 126",41.395488,2.177199,17.0,"C/ ROGER DE FLOR, 126",8013.0,27.0,"41.395487700000004,2.1771985000000003",el Fort Pienc,Eixample,Barcelona
10252,3,18,18,0,9,2019-10-01 00:01:22,IN_SERVICE,2019-10-01 00:05:10,20,2019-10-01,2,1,"C/ NÀPOLS, 82",41.394156,2.181331,11.0,"C/ NÀPOLS, 82",8013.0,27.0,"41.394155700000006,2.1813305",el Fort Pienc,Eixample,Barcelona
10253,4,14,14,0,4,2019-10-01 00:04:38,IN_SERVICE,2019-10-01 00:05:10,20,2019-10-01,2,1,"C/ RIBES, 13",41.393317,2.181248,8.0,"C/ RIBES, 13",8013.0,21.0,"41.39331729999999,2.1812482999999996",el Fort Pienc,Eixample,Barcelona
10254,5,36,35,1,2,2019-10-01 00:03:35,IN_SERVICE,2019-10-01 00:05:10,20,2019-10-01,2,1,"PG. LLUIS COMPANYS, 11 (ARC TRIOMF)",41.391103,2.180176,7.0,"PG. LLUIS COMPANYS, 11 (ARC TRIOMF)",8018.0,39.0,"41.3911035,2.1801763",Santa Caterina i la Ribera,Ciutat Vella,Barcelona


Next, we deleted all the stations that weren't located in Barcelona.

In [116]:
# Checking if some station were located outside BCN.
print(sum(df_clean['Municipi'] != "Barcelona"))

df_clean = df_clean[df_clean['Municipi'] == "Barcelona"]

# "Undo" operation of the groupby. Returns a "traditional-structured" data frame.
df_clean = df_clean.reset_index()

71075


For last, several columns were renamed to ease their reading and selection.

In [ ]:
df_clean = df_clean.rename(columns = {'num_bikes_available':'available_bikes','num_bikes_available_types.mechanical':'mechanical','num_bikes_available_types.ebike':'electrical'})
df_clean = df_clean.drop(columns = 'Municipi')
df_clean.head()

To end, we generated a CSV from our clean dataset and stored it in the cloud via *Google Drive* to use it on the visualizations creation process.

In [ ]:
df_clean.to_csv("CleanData.csv", index = False)
!cp CleanData.csv '/content/gdrive/My Drive/GCED/Q5/VI/Projecte 2'